In [1]:
import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
import pyvista as pv

In [2]:
# Keeping pandas from truncating long strings
pd.set_option('display.max_colwidth', 10000000)

In [3]:
# ensure correct individual name
root = 'c:\\Users\\ebuly\\OneDrive\\Documents\\WORK\\JRF_GaitAnalysis'
cwd = os.getcwd()
tmp = cwd.split('\\')[len(root.split('\\')):]
ind = 'P' + tmp[0][1:]
ind

'P08'

In [4]:
# transformed surfaces
path_to_bones = '4_tps-bones-muscles-updated/bones'
sacrum = pv.read(os.path.join(path_to_bones, 'sacrum.stl'))
r_pelvis = pv.read(os.path.join(path_to_bones, 'r_pelvis.stl'))
l_pelvis = pv.read(os.path.join(path_to_bones, 'l_pelvis.stl'))
femur_r = pv.read(os.path.join(path_to_bones, 'femur_r.stl'))
femur_l = pv.read(os.path.join(path_to_bones, 'femur_l.stl'))
tibia_r = pv.read(os.path.join(path_to_bones, 'tibia_r.stl'))
tibia_l = pv.read(os.path.join(path_to_bones, 'tibia_l.stl'))
fibula_r = pv.read(os.path.join(path_to_bones, 'fibula_r.stl'))
fibula_l = pv.read(os.path.join(path_to_bones, 'fibula_l.stl'))
patella_r = pv.read(os.path.join(path_to_bones, 'patella_r.stl'))
patella_l = pv.read(os.path.join(path_to_bones, 'patella_l.stl'))

In [5]:
# muscles and skin
pers_model = f'4_tps-bones-muscles-updated/{ind}_tps_fibres_skin_wrp_updated.osim'
gen_model = f"3a_osim_markers/scaled_model_{ind}.osim"

## pers
pers_model_tree=ET.parse(pers_model)
pers_model_root=pers_model_tree.getroot()

muscle_paths_temp = []
for PathPoint in pers_model_root.iter('PathPoint'):
    name = PathPoint.attrib['name']
    body = PathPoint.find('socket_parent_frame').text.split('/')[-1]
    location = [float(x) for x in PathPoint.find('location').text.split()]
    muscle_paths_temp.append({'name':name, 'body':body, 'location':location})
muscle_paths_pers_df = pd.DataFrame(muscle_paths_temp)
muscle_paths_pers_df = muscle_paths_pers_df.set_index('name')

skin_markers_temp = []
for marker in pers_model_root.iter('Marker'):
    name = marker.attrib['name']
    body = marker.find('socket_parent_frame').text.split('/')[-1]
    location = [float(x) for x in marker.find('location').text.split()]    
    skin_markers_temp.append({'name':name, 'body':body, 'location':location})
skin_markers_pers_df = pd.DataFrame(skin_markers_temp)
skin_markers_pers_df = skin_markers_pers_df.set_index('name')
## gen
gen_model_tree = ET.parse(gen_model)
gen_model_root = gen_model_tree.getroot()

muscle_paths_temp = []
for PathPoint in gen_model_root.iter('PathPoint'):
    name = PathPoint.attrib['name']
    body = PathPoint.find('socket_parent_frame').text.split('/')[-1]
    location = [float(x) for x in PathPoint.find('location').text.split()]
    muscle_paths_temp.append({'name':name, 'body':body, 'location':location})
muscle_paths_gen_df = pd.DataFrame(muscle_paths_temp)
muscle_paths_gen_df = muscle_paths_gen_df.set_index('name')

skin_markers_temp = []
for marker in gen_model_root.iter('Marker'):
    name = marker.attrib['name']
    body = marker.find('socket_parent_frame').text.split('/')[-1]
    location = [float(x) for x in marker.find('location').text.split()]    
    skin_markers_temp.append({'name':name, 'body':body, 'location':location})

skin_markers_gen_df = pd.DataFrame(skin_markers_temp)
skin_markers_gen_df = skin_markers_gen_df.set_index('name')

In [6]:
# bone markers
## pers
pers_bone_markers_df = pd.read_csv('tps_warping_results/after_MRI/markers_transformed.csv',index_col='name')
pers_bone_markers_df['location'] = pers_bone_markers_df['location'].apply(lambda x: [float(y) for y in x[1:-1].split()])
## gen
gen_bone_markers_xml = f"3a_osim_markers/bone_markers_{ind[1:]}.xml"
gen_bone_markers_tree=ET.parse(gen_bone_markers_xml)
gen_bone_markers_root = gen_bone_markers_tree.getroot()

temp = []

for marker in gen_bone_markers_root.iter('Marker'):
    name = marker.attrib['name']
    body = marker.find('socket_parent_frame').text.split('/')[-1]
    location = [float(x) for x in (marker.find('location').text).split()]
    temp.append({'name':name, 'location':location, 'body':body })
gen_bone_markers_df = pd.DataFrame(temp)
gen_bone_markers_df.set_index('name', inplace=True)

In [7]:
def process_segment_data(skin_markers_gen_df, skin_markers_pers_df, 
                         muscle_paths_gen_df, muscle_paths_pers_df, 
                         gen_bone_markers_df, pers_bone_markers_df, segment=None):
    def process_data(gen_df, pers_df, group, blacklist=None):
        gen_segment_df = gen_df[gen_df['body'] == segment]
        pers_segment_df = pers_df[pers_df['body'] == segment]
        
        if blacklist:
            gen_segment_df = gen_segment_df.drop(blacklist, errors='ignore')
            pers_segment_df = pers_segment_df.drop(blacklist, errors='ignore')

        item_names = list(gen_segment_df.index)
        pers_segment_df = pers_segment_df.loc[item_names]

        data = {
            'name': item_names * 2,
            'model': ['gen'] * len(item_names) + ['pers'] * len(item_names),
            'body': [segment] * 2 * len(item_names),
            'group': [group] * 2 * len(item_names),
            'location': gen_segment_df['location'].to_list() + pers_segment_df['location'].to_list()
        }
        values_gen = np.array(gen_segment_df['location'].to_list())
        values_pers = np.array(pers_segment_df['location'].to_list())
        
        return pd.DataFrame(data), item_names, values_gen, values_pers

    # Skin markers
    skin_df, skin_names, skin_gen_values, skin_pers_values = process_data(
        skin_markers_gen_df, skin_markers_pers_df, 'skin'
    )

    # Muscle paths
    muscle_df, muscle_names, muscle_gen_values, muscle_pers_values = process_data(
        muscle_paths_gen_df, muscle_paths_pers_df, 'muscles'
    )

    # Bone markers
    skin_blacklist = skin_names
    bone_df, bone_names, bone_gen_values, bone_pers_values = process_data(
        gen_bone_markers_df, pers_bone_markers_df, 'bone', blacklist=skin_blacklist
    )

    # Combine all dataframes
    segment_df = pd.concat([skin_df, muscle_df, bone_df], ignore_index=True)

    return {
        f'df': segment_df,
        f'skin_marker_names': skin_names,
        f'skin_gen_values': skin_gen_values,
        f'skin_pers_values': skin_pers_values,
        f'muscle_names': muscle_names,
        f'muscle_gen_values': muscle_gen_values,
        f'muscle_pers_values': muscle_pers_values,
        f'bone_marker_names': bone_names,
        f'bone_gen_values': bone_gen_values,
        f'bone_pers_values': bone_pers_values,
    }


## Pelvis

In [8]:
pv_res = process_segment_data(skin_markers_gen_df, skin_markers_pers_df, 
                         muscle_paths_gen_df, muscle_paths_pers_df, 
                         gen_bone_markers_df, pers_bone_markers_df, segment='pelvis')

In [9]:
# plot pelvis shape and pelvis muscle points
plotter = pv.Plotter(shape=(1, 2))
actor = plotter.add_mesh(l_pelvis, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(r_pelvis, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(sacrum, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(pv_res['bone_gen_values']),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(pv_res['bone_pers_values']),  color='blue', point_size=10.0)
for i, value in enumerate(pv_res['bone_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, pv_res['bone_pers_values'][i]),  color='black')

plotter.subplot(0, 1)
actor = plotter.add_mesh(l_pelvis, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(r_pelvis, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(sacrum, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(pv_res['skin_gen_values']),  color='grey', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(pv_res['skin_pers_values']),  color='red', point_size=10.0)
for i, value in enumerate(pv_res['skin_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, pv_res['skin_pers_values'][i]),  color='grey')


actor = plotter.add_mesh(pv.PolyData(pv_res['muscle_gen_values']),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(pv_res['muscle_pers_values']),  color='darkgreen', point_size=10.0)
for i, value in enumerate(pv_res['muscle_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, pv_res['muscle_pers_values'][i]),  color='black')

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:55548/index.html?ui=P_0x252a3e90550_0&reconnect=auto" class="pyvis…

## tibia_r

In [10]:
tb_r_res = process_segment_data(skin_markers_gen_df, skin_markers_pers_df, 
                         muscle_paths_gen_df, muscle_paths_pers_df, 
                         gen_bone_markers_df, pers_bone_markers_df, segment='tibia_r')

In [11]:
# plot tibia_r shape and tibia_r muscle points
plotter = pv.Plotter(shape=(1, 2))

## labels for muscles:
poly_tibia_r_muscle_pers_values = pv.PolyData(tb_r_res['muscle_pers_values'])
poly_tibia_r_muscle_pers_values["Labels"] = tb_r_res['muscle_names']

actor = plotter.add_mesh(tibia_r, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(fibula_r, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tb_r_res['bone_gen_values']),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(tb_r_res['bone_pers_values']),  color='blue', point_size=10.0)
for i, value in enumerate(tb_r_res['bone_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, tb_r_res['bone_pers_values'][i]),  color='black')

plotter.subplot(0, 1)
actor = plotter.add_mesh(tibia_r, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(fibula_r, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(tb_r_res['skin_gen_values']),  color='grey', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(tb_r_res['skin_pers_values']),  color='red', point_size=10.0)
for i, value in enumerate(tb_r_res['skin_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, tb_r_res['skin_pers_values'][i]),  color='grey')


actor = plotter.add_mesh(pv.PolyData(tb_r_res['muscle_gen_values']),  color='black', point_size=5.0)

actor = plotter.add_point_labels(poly_tibia_r_muscle_pers_values, "Labels", point_size=10, font_size=12)
actor = plotter.add_mesh(pv.PolyData(tb_r_res['muscle_pers_values']),  color='darkgreen', point_size=10.0)
for i, value in enumerate(tb_r_res['muscle_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, tb_r_res['muscle_pers_values'][i]),  color='black')

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:55548/index.html?ui=P_0x252a6d64090_1&reconnect=auto" class="pyvis…

## tibia_l

In [12]:
tb_l_res = process_segment_data(skin_markers_gen_df, skin_markers_pers_df, 
                         muscle_paths_gen_df, muscle_paths_pers_df, 
                         gen_bone_markers_df, pers_bone_markers_df, segment='tibia_l')

## femur_r

In [13]:
fm_r_res = process_segment_data(skin_markers_gen_df, skin_markers_pers_df, 
                         muscle_paths_gen_df, muscle_paths_pers_df, 
                         gen_bone_markers_df, pers_bone_markers_df, segment='femur_r')

In [14]:
# labels for muscles:
poly_femur_r_muscle_pers_values = pv.PolyData(fm_r_res['muscle_pers_values'])
poly_femur_r_muscle_pers_values["Labels"] = fm_r_res['muscle_names']
# plot femur_r shape and femur_r muscle points
plotter = pv.Plotter(shape=(1, 2))

actor = plotter.add_mesh(femur_r, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(fm_r_res['bone_gen_values']),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(fm_r_res['bone_pers_values']),  color='blue', point_size=10.0)
for i, value in enumerate(fm_r_res['bone_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, fm_r_res['bone_pers_values'][i]),  color='black')

plotter.subplot(0, 1)
actor = plotter.add_mesh(femur_r, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(fm_r_res['skin_gen_values']),  color='grey', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(fm_r_res['skin_pers_values']),  color='red', point_size=10.0)
for i, value in enumerate(fm_r_res['skin_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, fm_r_res['skin_pers_values'][i]),  color='grey')


actor = plotter.add_mesh(pv.PolyData(fm_r_res['muscle_gen_values']),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(fm_r_res['muscle_pers_values']),  color='darkgreen', point_size=10.0)
actor = plotter.add_point_labels(poly_femur_r_muscle_pers_values, "Labels", point_size=10, font_size=12)
for i, value in enumerate(fm_r_res['muscle_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, fm_r_res['muscle_pers_values'][i]),  color='black')

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:55548/index.html?ui=P_0x252e339da10_2&reconnect=auto" class="pyvis…

## femur_l

In [15]:
fm_l_res = process_segment_data(skin_markers_gen_df, skin_markers_pers_df, 
                         muscle_paths_gen_df, muscle_paths_pers_df, 
                         gen_bone_markers_df, pers_bone_markers_df, segment='femur_l')

## patella_r

In [16]:
pt_r_res = process_segment_data(skin_markers_gen_df, skin_markers_pers_df, 
                         muscle_paths_gen_df, muscle_paths_pers_df, 
                         gen_bone_markers_df, pers_bone_markers_df, segment='patella_r')

In [17]:
# labels for muscles:
poly_muscle_pers_values = pv.PolyData(pt_r_res['muscle_pers_values'])
poly_muscle_pers_values["Labels"] = pt_r_res['muscle_names']

plotter = pv.Plotter(shape=(1, 2))

actor = plotter.add_mesh(patella_r, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(pt_r_res['bone_gen_values']),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(pt_r_res['bone_pers_values']),  color='blue', point_size=10.0)
for i, value in enumerate(pt_r_res['bone_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, pt_r_res['bone_pers_values'][i]),  color='black')

plotter.subplot(0, 1)
actor = plotter.add_mesh(patella_r, color='lightblue', opacity=0.2, show_edges=True)
actor = plotter.add_mesh(pv.PolyData(pt_r_res['muscle_gen_values']),  color='black', point_size=5.0)
actor = plotter.add_mesh(pv.PolyData(pt_r_res['muscle_pers_values']),  color='darkgreen', point_size=10.0)
actor = plotter.add_point_labels(poly_muscle_pers_values, "Labels", point_size=10, font_size=12)
for i, value in enumerate(pt_r_res['muscle_gen_values']):
    actor = plotter.add_mesh(pv.Line(value, pt_r_res['muscle_pers_values'][i]),  color='black')

plotter.show(jupyter_backend='trame')

Widget(value='<iframe src="http://localhost:55548/index.html?ui=P_0x252a6f6c350_3&reconnect=auto" class="pyvis…

## patella_l

In [18]:
pt_l_res = process_segment_data(skin_markers_gen_df, skin_markers_pers_df, 
                         muscle_paths_gen_df, muscle_paths_pers_df, 
                         gen_bone_markers_df, pers_bone_markers_df, segment='patella_l')

In [19]:
full_df = pd.concat([pv_res['df'], fm_r_res['df'], fm_l_res['df'],tb_r_res['df'], tb_l_res['df'], pt_r_res['df'], pt_l_res['df']])

In [20]:
full_df.to_csv(f'C:\\Users\\User\\OneDrive\\Documents\\WORK\\JRF_GaitAnalysis\\analyze\\{ind}_data.csv')